In [1]:
# pylint: disable=invalid-name
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction
print("Imports Successful")

Imports Successful


In [ ]:
def shors_algorithm_random(N, n_count=8, max_trials=1024):
    """
    Attempts to factor N using Shor's Algorithm with randomly chosen 'a'.
    
    Args:
        N (int): Composite number to factor.
        n_count (int): Number of qubits for phase estimation.
        max_trials (int): Number of random bases 'a' to try.

    Returns:
        tuple: (factor1, factor2, a, r) if successful, else (None, None, None, None).
    """
    from math import gcd, ceil, log2
    from fractions import Fraction
    import random
    from qiskit import QuantumCircuit, transpile
    from qiskit.circuit.library import QFT
    from qiskit_aer import AerSimulator

    def c_amodN_classical(a, power, N, n_bits):
        """Creates a classically hardcoded controlled modular multiplication gate."""
        m = pow(a, power, N)
        qc = QuantumCircuit(n_bits)
        binary = format(m, f"0{n_bits}b")
        for i, bit in enumerate(reversed(binary)):
            if bit == '1':
                qc.x(i)
        return qc.to_gate(label=f"{a}^{power} mod {N}").control()

    def modular_exponentiation_circuit(a, N, n_count):
        """Constructs modular exponentiation circuit for given a, N."""
        n_bits = ceil(log2(N))
        qc = QuantumCircuit(n_count + n_bits)
        qc.x(n_count)  # Initialize target to |1⟩
        for i in range(n_count):
            gate = c_amodN_classical(a, 2**i, N, n_bits)
            qc.append(gate, [i] + list(range(n_count, n_count + n_bits)))
        return qc

    for trial in range(max_trials):
        a = random.randint(2, N - 1)
        if gcd(a, N) != 1:
            factor = gcd(a, N)
            print(f"✔ Found classically: {factor} * {N//factor} = {N} (a = {a})")
            return factor, N // factor, a, None

        print(f"Trial {trial+1}: Trying a = {a}")
        n_bits = ceil(log2(N))
        modexp = modular_exponentiation_circuit(a, N, n_count)

        qc = QuantumCircuit(n_count + n_bits, n_count)
        qc.h(range(n_count))
        qc.x(n_count)
        qc.compose(modexp, inplace=True)
        qc.append(QFT(n_count, inverse=True), range(n_count))
        qc.measure(range(n_count), range(n_count))

        backend = AerSimulator()
        tqc = transpile(qc, backend)
        result = backend.run(tqc, shots=1, memory=True).result()
        measured = result.get_memory()[0]
        phase = int(measured, 2) / 2**n_count
        frac = Fraction(phase).limit_denominator(N)
        r = frac.denominator
        print(f"  ↪ Estimated r = {r}")

        if r % 2 == 0 and pow(a, r // 2, N) not in [1, N - 1]:
            x = pow(a, r // 2, N)
            factor1 = gcd(x - 1, N)
            factor2 = gcd(x + 1, N)
            if factor1 not in [1, N] and factor2 not in [1, N]:
                print(f"Success! {factor1} * {factor2} = {N} (a = {a}, r = {r})")
                return factor1, factor2, a, r

        print("  ✖ Failed for this a.")

    print("No non-trivial factors found after all trials.")
    return None, None, None, None


In [17]:
f1, f2, a, r = shors_algorithm_random(3127)
print(f"\nResult: {f1} * {f2} = 3127 (a = {a}, r = {r})")


Trial 1: Trying a = 1053
  ↪ Estimated r = 128
  ✖ Failed for this a.
✔ Found classically: 59 * 53 = 3127 (a = 177)

Result: 59 * 53 = 3127 (a = 177, r = None)


In [22]:
f1, f2, a, r = shors_algorithm_random(167659)
print(f"\nResult: {f1} * {f2} = 167659 (a = {a}, r = {r})")

Trial 1: Trying a = 111163
  ↪ Estimated r = 256
  ✖ Failed for this a.
Trial 2: Trying a = 102571
  ↪ Estimated r = 256
  ✖ Failed for this a.
Trial 3: Trying a = 141227
  ↪ Estimated r = 32
  ✖ Failed for this a.
Trial 4: Trying a = 123572
  ↪ Estimated r = 256
  ✖ Failed for this a.
Trial 5: Trying a = 151735
  ↪ Estimated r = 32
  ✖ Failed for this a.
Trial 6: Trying a = 68011
  ↪ Estimated r = 256
  ✖ Failed for this a.
Trial 7: Trying a = 9274
  ↪ Estimated r = 32
  ✖ Failed for this a.
Trial 8: Trying a = 154896
  ↪ Estimated r = 4
  ✖ Failed for this a.
Trial 9: Trying a = 20342
  ↪ Estimated r = 256
  ✖ Failed for this a.
Trial 10: Trying a = 72586
  ↪ Estimated r = 64
  ✖ Failed for this a.
Trial 11: Trying a = 41308
  ↪ Estimated r = 64
  ✖ Failed for this a.
Trial 12: Trying a = 151555
  ↪ Estimated r = 128
  ✖ Failed for this a.
Trial 13: Trying a = 54093
  ↪ Estimated r = 256
  ✖ Failed for this a.
Trial 14: Trying a = 101889
  ↪ Estimated r = 256
  ✖ Failed for this a.
T

In [24]:
shors_algorithm_real(15)

✔ Found classically: 3 * 5 = 15 (a = 12)


(3, 5, 12, None)

In [27]:
shors_algorithm_real(21)

Trial 1: Trying a = 16
  ↪ Estimated r = 21
  ✖ Failed for this a.
✔ Found classically: 3 * 7 = 21 (a = 12)


(3, 7, 12, None)

In [ ]:
import numpy as np
from math import gcd, ceil, log2
from fractions import Fraction
import random

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT, CDKMRippleCarryAdder

def quantum_modular_addition(a, N, n_bits):
    """
    Constructs a quantum circuit that performs (x + a) mod N using a ripple-carry adder.

    Parameters:
        a (int): The constant to add.
        N (int): The modulus (used for sizing purposes).
        n_bits (int): Number of bits for the input/target registers.

    Returns:
        QuantumCircuit: A quantum circuit performing modular addition of a.
    """
    adder = CDKMRippleCarryAdder(n_bits)
    expected_qubits = adder.num_qubits
    qc = QuantumCircuit(expected_qubits)

    a_bin = format(a, f"0{n_bits}b")
    for i, bit in enumerate(reversed(a_bin)):
        if bit == '1':
            qc.x(i + n_bits)

    qc.append(adder.to_gate(), list(range(expected_qubits)))
    return qc

def controlled_modular_exponentiation(a, N, n_count):
    """
    Constructs a modular exponentiation circuit controlled by n_count qubits.
    Computes (a^x mod N) where x is encoded in the control register.

    Parameters:
        a (int): Base for exponentiation.
        N (int): Modulus.
        n_count (int): Number of qubits in the counting (phase estimation) register.

    Returns:
        QuantumCircuit: A circuit that performs modular exponentiation under control.
    """
    n_bits = ceil(log2(N))
    adder = CDKMRippleCarryAdder(n_bits)
    work_qubits = adder.num_qubits
    qc = QuantumCircuit(n_count + work_qubits)
    qc.x(n_count)  # Initialize |1> in target register

    for i in range(n_count):
        exponent = 2**i
        mult = pow(a, exponent, N)
        mod_add = quantum_modular_addition(mult, N, n_bits)
        control = [i]
        targets = list(range(n_count, n_count + work_qubits))
        qc.append(mod_add.to_gate().control(1), control + targets)

    return qc

def shors_algorithm_real(N, n_count=8, max_trials=20):
    """
    Executes Shor's Algorithm to factor a composite integer N using quantum phase estimation.

    Parameters:
        N (int): The composite integer to factor.
        n_count (int): Number of qubits in the phase estimation (counting) register.
        max_trials (int): Maximum number of random base 'a' trials.

    Returns:
        tuple: (factor1, factor2, a, r)
            factor1, factor2: Non-trivial factors of N
            a: The randomly chosen base that led to successful factoring
            r: The estimated order of a mod N
        If unsuccessful, returns (None, None, None, None).
    """
    for trial in range(max_trials):
        a = random.randint(2, N - 1)
        if gcd(a, N) != 1:
            factor = gcd(a, N)
            print(f"✔ Found classically: {factor} * {N // factor} = {N} (a = {a})")
            return factor, N // factor, a, None

        print(f"Trial {trial+1}: Trying a = {a}")
        n_bits = ceil(log2(N))
        modexp = controlled_modular_exponentiation(a, N, n_count)
        total_qubits = modexp.num_qubits

        qc = QuantumCircuit(total_qubits, n_count)
        qc.h(range(n_count))
        qc.compose(modexp, inplace=True)
        qc.append(QFT(n_count, inverse=True).to_gate(), range(n_count))
        qc.measure(range(n_count), range(n_count))

        backend = AerSimulator()
        tqc = transpile(qc, backend)
        result = backend.run(tqc, shots=1, memory=True).result()
        measured = result.get_memory()[0]
        phase = int(measured, 2) / 2**n_count
        frac = Fraction(phase).limit_denominator(N)
        r = frac.denominator

        print(f"  ↪ Estimated r = {r}")

        if r % 2 == 0 and pow(a, r // 2, N) not in [1, N - 1]:
            x = pow(a, r // 2, N)
            factor1 = gcd(x - 1, N)
            factor2 = gcd(x + 1, N)
            if factor1 not in [1, N] and factor2 not in [1, N]:
                print(f"Success! {factor1} * {factor2} = {N} (a = {a}, r = {r})")
                return factor1, factor2, a, r

        print("  ✖ Failed for this a.")

    print("No non-trivial factors found after all trials.")
    return None, None, None, None
import numpy as np
from math import gcd, ceil, log2
from fractions import Fraction
import random

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT, CDKMRippleCarryAdder

def quantum_modular_addition(a, N, n_bits):
    """
    Constructs a quantum circuit that performs (x + a) mod N using a ripple-carry adder.

    Parameters:
        a (int): The constant to add.
        N (int): The modulus (used for sizing purposes).
        n_bits (int): Number of bits for the input/target registers.

    Returns:
        QuantumCircuit: A quantum circuit performing modular addition of a.
    """
    adder = CDKMRippleCarryAdder(n_bits)
    expected_qubits = adder.num_qubits
    qc = QuantumCircuit(expected_qubits)

    a_bin = format(a, f"0{n_bits}b")
    for i, bit in enumerate(reversed(a_bin)):
        if bit == '1':
            qc.x(i + n_bits)

    qc.append(adder.to_gate(), list(range(expected_qubits)))
    return qc

def controlled_modular_exponentiation(a, N, n_count):
    """
    Constructs a modular exponentiation circuit controlled by n_count qubits.
    Computes (a^x mod N) where x is encoded in the control register.

    Parameters:
        a (int): Base for exponentiation.
        N (int): Modulus.
        n_count (int): Number of qubits in the counting (phase estimation) register.

    Returns:
        QuantumCircuit: A circuit that performs modular exponentiation under control.
    """
    n_bits = ceil(log2(N))
    adder = CDKMRippleCarryAdder(n_bits)
    work_qubits = adder.num_qubits
    qc = QuantumCircuit(n_count + work_qubits)
    qc.x(n_count)  # Initialize |1> in target register

    for i in range(n_count):
        exponent = 2**i
        mult = pow(a, exponent, N)
        mod_add = quantum_modular_addition(mult, N, n_bits)
        control = [i]
        targets = list(range(n_count, n_count + work_qubits))
        qc.append(mod_add.to_gate().control(1), control + targets)

    return qc

def shors_algorithm_real(N, n_count=8, max_trials=20):
    """
    Executes Shor's Algorithm to factor a composite integer N using quantum phase estimation.

    Parameters:
        N (int): The composite integer to factor.
        n_count (int): Number of qubits in the phase estimation (counting) register.
        max_trials (int): Maximum number of random base 'a' trials.

    Returns:
        tuple: (factor1, factor2, a, r)
            factor1, factor2: Non-trivial factors of N
            a: The randomly chosen base that led to successful factoring
            r: The estimated order of a mod N
        If unsuccessful, returns (None, None, None, None).
    """
    for trial in range(max_trials):
        a = random.randint(2, N - 1)
        if gcd(a, N) != 1:
            factor = gcd(a, N)
            print(f"✔ Found classically: {factor} * {N // factor} = {N} (a = {a})")
            return factor, N // factor, a, None

        print(f"Trial {trial+1}: Trying a = {a}")
        n_bits = ceil(log2(N))
        modexp = controlled_modular_exponentiation(a, N, n_count)
        total_qubits = modexp.num_qubits

        qc = QuantumCircuit(total_qubits, n_count)
        qc.h(range(n_count))
        qc.compose(modexp, inplace=True)
        qc.append(QFT(n_count, inverse=True).to_gate(), range(n_count))
        qc.measure(range(n_count), range(n_count))

        backend = AerSimulator()
        tqc = transpile(qc, backend)
        result = backend.run(tqc, shots=1, memory=True).result()
        measured = result.get_memory()[0]
        phase = int(measured, 2) / 2**n_count
        frac = Fraction(phase).limit_denominator(N)
        r = frac.denominator

        print(f"  ↪ Estimated r = {r}")

        if r % 2 == 0 and pow(a, r // 2, N) not in [1, N - 1]:
            x = pow(a, r // 2, N)
            factor1 = gcd(x - 1, N)
            factor2 = gcd(x + 1, N)
            if factor1 not in [1, N] and factor2 not in [1, N]:
                print(f"Success! {factor1} * {factor2} = {N} (a = {a}, r = {r})")
                return factor1, factor2, a, r

        print("  ✖ Failed for this a.")

    print("No non-trivial factors found after all trials.")
    return None, None, None, None
